In [66]:
import respy as rp
import pandas as pd
import numpy as np

from auxiliary import *

pd.set_option('display.max_rows', 1000)

In [68]:

num_occupations, n_periods = 3, 3

params_occ, options_occ = scaling_model_specification("kw_97_basic", num_periods=n_periods, num_occupations=num_occupations)

In [69]:
params_occ

value         sd  \
category              name                                                     
delta                 delta                              0.787000     0.0048   
initial_exp_school_10 probability                        0.671500     0.0500   
initial_exp_school_11 probability                        0.075000     0.0500   
initial_exp_school_7  probability                        0.009500     0.0500   
initial_exp_school_8  probability                        0.042200     0.0500   
initial_exp_school_9  probability                        0.201800     0.0500   
maximum_exp           school                            20.000000        NaN   
nonpec_home           constant                       16887.000000   413.0000   
                      type_1                           215.000000   377.0000   
                      type_2                        -16966.000000   542.0000   
                      type_3                        -13128.000000  1000.0000   
nonpec_school         co_graduate                   -26357.000000   737.0000   
                      constant                       43948.000000   850.0000   
                      hs_graduate                    -2983.000000   156.0000   
                      type_1                        -26352.000000   757.0000   
                      type_2                        -30541.000000   754.0000   
                      type_3                           226.000000   594.0000   
type_1                at_least_ten_years_school          2.433570     0.2000   
                      up_to_nine_years_school            0.313614     0.0250   
type_2                at_least_ten_years_school          2.536240     0.2000   
                      up_to_nine_years_school            1.052250     0.0400   
type_3                at_least_ten_years_school         -0.159780     0.0500   
                      up_to_nine_years_school           -0.736924     0.1000   
wage_aa               constant                           8.937000     0.0152   
                      exp_blue_collar                    0.022500     0.0008   
                      exp_military                       0.013100     0.0131   
                      exp_school                         0.070000     0.0018   
                      exp_white_collar                   0.027000     0.0012   
                      exp_white_collar_square           -0.042900     0.0032   
wage_bb               constant                           8.937000     0.0152   
                      exp_blue_collar                    0.022500     0.0008   
                      exp_military                       0.013100     0.0131   
                      exp_school                         0.070000     0.0018   
                      exp_white_collar                   0.027000     0.0012   
                      exp_white_collar_square           -0.042900     0.0032   
wage_blue_collar      constant                           8.915600     0.0126   
                      exp_blue_collar                    0.142400     0.0011   
                      exp_blue_collar_square            -0.177400     0.0041   
                      exp_military                       0.102100     0.0021   
                      exp_school                         0.018900     0.0014   
                      exp_white_collar                   0.067400     0.0674   
                      type_1                             0.299600     0.0094   
                      type_2                            -0.122300     0.0079   
                      type_3                             0.075600     0.0058   
wage_cc               constant                           8.937000     0.0152   
                      exp_blue_collar                    0.022500     0.0008   
                      exp_military                       0.013100     0.0131   
                      exp_school                         0.070000     0.0018   
                      exp_white_collar                   0.0